In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping , ModelCheckpoint
from tensorflow.keras.models import load_model
from datetime import datetime

In [2]:
data_dir = '/Users/irk2w/Desktop/T5/sounds_final '
classes = ['Civil', 'Police', 'Trafic', 'ambulance']

In [5]:
def extract_features(file_path, target_shape=(128, 128)):
    audio_data, sample_rate = librosa.load(file_path, sr=None)

    # Data Augmentation
    pitch_shifted = librosa.effects.pitch_shift(audio_data, sr=sample_rate, n_steps=4)
    time_stretched = librosa.effects.time_stretch(audio_data, rate=1.5)

    # Original Features
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    # Augmented Features
    mel_spectrogram_pitch = librosa.feature.melspectrogram(y=pitch_shifted, sr=sample_rate)
    mel_spectrogram_stretch = librosa.feature.melspectrogram(y=time_stretched, sr=sample_rate)

    # Resizing
    mel_spectrogram_resized = resize(mel_spectrogram, target_shape)
    mel_spectrogram_pitch_resized = resize(mel_spectrogram_pitch, target_shape)
    mel_spectrogram_stretch_resized = resize(mel_spectrogram_stretch, target_shape)

    return mel_spectrogram_resized, mel_spectrogram_pitch_resized, mel_spectrogram_stretch_resized


In [6]:
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                original, pitch, stretch = extract_features(file_path, target_shape)
                # Append original features
                data.append(original)
                labels.append(i)
                # Append augmented features
                data.append(pitch)
                labels.append(i)
                data.append(stretch)
                labels.append(i)

    return np.array(data), np.array(labels)

In [7]:
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
data, l = load_and_preprocess_data(data_dir, classes)

In [ ]:
pd.Series(l).value_counts()

In [8]:
model = Sequential()
model.add(Dense(256, input_shape=(128, 128), activation='relu'))  # Adjust the input_shape according to your data
model.add(Dropout(0.5))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(LSTM(64))
model.add(Dense(len(classes), activation='softmax'))

In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [10]:
# Display model architecture summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128, 256)          33024     
                                                                 
 dropout (Dropout)           (None, 128, 256)          0         
                                                                 
 dense_1 (Dense)             (None, 128, 192)          49344     
                                                                 
 dropout_1 (Dropout)         (None, 128, 192)          0         
                                                                 
 dense_2 (Dense)             (None, 128, 128)          24704     
                                                                 
 dropout_2 (Dropout)         (None, 128, 128)          0         
                                                                 
 dense_3 (Dense)             (None, 128, 64)           8

In [12]:
# Calculate pre-training accuracy
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

In [13]:
# Print pre-training accuracy
print("Pre-training accuracy: %.4f%%" % accuracy)

Pre-training accuracy: 28.0000%


In [14]:
# Train the model
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.h5',
                               verbose=1, save_best_only=True)

In [16]:
start = datetime.now()

In [17]:
model.fit(X_train, y_train, batch_size=256, epochs=400, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/400
2/2 [==============================] - ETA: 0s - loss: 1.3864 - accuracy: 0.2487
Epoch 1: val_loss improved from inf to 1.38686, saving model to saved_models/weights.best.basic_mlp.h5
2/2 [==============================] - 1s 219ms/step - loss: 1.3864 - accuracy: 0.2487 - val_loss: 1.3869 - val_accuracy: 0.1900
Epoch 2/400
1/2 [==============>...............] - ETA: 0s - loss: 1.3856 - accuracy: 0.2695

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - ETA: 0s - loss: 1.3855 - accuracy: 0.2764
Epoch 2: val_loss did not improve from 1.38686
2/2 [==============================] - 0s 154ms/step - loss: 1.3855 - accuracy: 0.2764 - val_loss: 1.3887 - val_accuracy: 0.1900
Epoch 3/400
2/2 [==============================] - ETA: 0s - loss: 1.3848 - accuracy: 0.2764
Epoch 3: val_loss did not improve from 1.38686
2/2 [==============================] - 0s 148ms/step - loss: 1.3848 - accuracy: 0.2764 - val_loss: 1.3911 - val_accuracy: 0.1900
Epoch 4/400
2/2 [==============================] - ETA: 0s - loss: 1.3840 - accuracy: 0.2764
Epoch 4: val_loss did not improve from 1.38686
2/2 [==============================] - 0s 147ms/step - loss: 1.3840 - accuracy: 0.2764 - val_loss: 1.3932 - val_accuracy: 0.1900
Epoch 5/400
2/2 [==============================] - ETA: 0s - loss: 1.3840 - accuracy: 0.2764
Epoch 5: val_loss did not improve from 1.38686
2/2 [==============================] - 0s 153ms/step - loss: 1.38

In [ ]:
duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
# Evaluating the model on the training and testing set
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

In [ ]:
model.save('911_class_test.h5')

In [ ]:
from tensorflow.keras.models import load_model

def predict_audio_class(file_path, model, classes):
    # Assuming the extract_features function is defined as above
    features = extract_features(file_path)[0]  # Use the original features for prediction
    features = np.expand_dims(features, axis=0)  # Reshaping to match model input

    # Make the prediction
    predictions = model.predict(features)
    predicted_class = np.argmax(predictions, axis=1)
    return classes[predicted_class[0]]

# Usage
#model = load_model("/Users/irk2w/Desktop/T5/911_model_don't_touch/911_class_test.h5")  # Load your model
audio_file_path = '/Users/irk2w/Downloads/civil1.wav'
predicted_class_name = predict_audio_class(audio_file_path, model, classes)
print(f"The model predicts that the audio file is a {predicted_class_name} sound.")

In [ ]:
confusion_matrix(y_train.argmax(axis = 1), model.predict(X_train).argmax(axis = 1))

In [ ]:
confusion_matrix(y_test.argmax(axis = 1), model.predict(X_test).argmax(axis = 1))

In [ ]:
len(y_train.argmax(axis = 1))

In [ ]:
len(y_train)